### Download data
This script is made to obtain Sentinel-2 satellite images for a given time period and area based on a JSON.
There are many ways to obtain S2 images, but this downloads the original JP2 files where you can easily work them over with ESA Snap tool. 

If you use this script, remember to define the directory you wish to save it to and provide your scihub login credentials.

In [2]:
#%% Imports
import json
import numpy as np
import zipfile
import os
import time
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import datetime
from multiprocessing.pool import ThreadPool
from re import sub

In [6]:
setdir = "D:/EO data/OPTICAL/Vegetation_ML/satellite_data/"

# Login information
api = SentinelAPI('YourUserName','YourPassword', 'https://scihub.copernicus.eu/dhus')

# Load in the geojson file with the coordinates
region = geojson_to_wkt(read_geojson('../../Optical/Vegetation_ML/json/broby_allested.geojson'))


# Time span definition
date = ['20220710','20220712']

# Which satellite is it. Outcomment the wrong one. For saving path.
s=str('S2')


# Extraction info
satdata = api.query(region,
                    date=(date[0], date[1]),
                    platformname='Sentinel-2',
                    processinglevel = 'Level-2A',
                    cloudcoverpercentage=(0,12))

# Converting to a geoJSON
satjson = api.to_geojson(satdata)

# Creating a backup
bu_images_json = satjson


#%%

# Listing name of the images being downloaded and file sizes
print("")
print("Executing script for download of Sentinel-2 files")
print("")
print("=================================================")
print("Listing file name and size:")
print("")
total = 0
for n in range(0, np.size(bu_images_json["features"])):
    print("Image name: ",json.dumps(bu_images_json["features"][n]["properties"]["title"]))
    print("File size: ",json.dumps(bu_images_json["features"][n]["properties"]["size"]))
    total += float(sub('[^0-9.]','',json.dumps(bu_images_json["features"][n]["properties"]["size"])))
print("")
print("Found", n+1, "files available for download")
print("Total amount to download: ",round(total, 1), "MB")


#%% Executing this part will initiate download and will take time

print("")    
print("=================================================")
print("Commencing download of", n+1, "files")
print("")
# Setting up download definition
def _download_images(n):
    try:
        print("Downloading file nr.:", n + 1, "of", np.size(bu_images_json["features"]))
        api.download(bu_images_json["features"][n]["properties"]["id"], directory_path=setdir+s+'/packed')
        time.sleep(3) 
    except:
        print("Download error detected: ", bu_images_json["features"][n]["properties"]["id"])
        time.sleep(2)
        pass

# Establishing time on download
startTime = datetime.now()

# Counting how many images is scheduled to be downloaded
n = np.size(bu_images_json["features"])

# Initiating the download
multidownload = ThreadPool(processes=2) #Can only download 2 images at once
multidownload.map(_download_images, (filename for filename in range(n)))
multidownload.close()

#%% Sorting the zipped content into appropriate folders
print("")    
print("=================================================")
print("Unzipping", n+1, "files")
print("")
folders = os.listdir(setdir+s+"/packed")
folders = [f for f in folders if '.zip' in f]
start = time.time()
# Now unwrapping the images and moving them into appropriate folders.
for f in folders:
    try:
        if 'L1C' in f:
            print("Unpacking file:", f)
            with zipfile.ZipFile(setdir+s+"/packed/"+f,"r") as zip_ref:
                zip_ref.extractall(setdir+s+"/unpacked/L1C")
            print("Succesfully unwrapped:",f)
            print("Time elapsed: ",round(time.time()-start,1), "seconds")
            print("")
        elif 'L2A' in f:
            print("Unpacking file:", f)
            with zipfile.ZipFile(setdir+s+"/packed/"+f,"r") as zip_ref:
                zip_ref.extractall(setdir+s+"/L2C")
            print("Succesfully unwrapped:",f)
            print("Time elapsed: ",round(time.time()-start,1), "seconds")
            print("")
    except:
        print("Unwrap error with file:", f)
        pass

print("=========================================")
print("Download and extraction of files complete")
print("=========================================")
print("")


Executing script for download of Sentinel-2 files

Listing file name and size:

Image name:  "S2B_MSIL2A_20220711T103629_N0400_R008_T32UNG_20220711T121934"
File size:  "1.02 GB"

Found 1 files available for download
Total amount to download:  1.0 MB

Commencing download of 1 files


Unzipping 2 files

Unpacking file: S2B_MSIL1C_20220711T103629_N0400_R008_T32UNG_20220711T112428.zip
Succesfully unwrapped: S2B_MSIL1C_20220711T103629_N0400_R008_T32UNG_20220711T112428.zip
Time elapsed:  2.8 seconds

Unpacking file: S2B_MSIL2A_20220711T103629_N0400_R008_T32UNG_20220711T121934.zip
Succesfully unwrapped: S2B_MSIL2A_20220711T103629_N0400_R008_T32UNG_20220711T121934.zip
Time elapsed:  6.8 seconds

Download and extraction of files complete

